In [ ]:
import pandas as pd

from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.project import get_project
from autumn.projects.sm_sir.australia.northern_territory.project import get_ts_date_indexes

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
region = "northern_territory"
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline

In [ ]:
model = p.run_baseline_model(baseline_params)
derived_df = model.get_derived_outputs_df()

In [ ]:
ts_set_dates = get_ts_date_indexes(p.ts_set, COVID_BASE_DATETIME)

In [ ]:
indicator = "notifications"
comparison_df = pd.DataFrame({
    "modelled": derived_df[indicator],
    "data": ts_set_dates[indicator]
})

In [ ]:
comparison_df.plot()